In [ ]:
%matplotlib inline

Create Sphere Mesh Multiple Ways {#create_sphere_example}
================================

This example shows how to create meshes in different ways.


In [ ]:
from __future__ import annotations

import numpy as np

import pyvista as pv

Simple Sphere
=============

The quickest method to get a Sphere mesh is to use
`pyvista.Sphere`{.interpreted-text role="func"}.


In [ ]:
mesh = pv.Sphere()
mesh.plot(show_edges=True)

This gives an `pyvista.PolyData`{.interpreted-text role="class"} mesh,
i.e. a 2D surface.


In [ ]:
mesh

In this case, it is
`manifold <pyvista.PolyData.is_manifold>`{.interpreted-text role="func"}
and encloses a volume. To demonstrate this, there are no boundaries on
the mesh as indicated by no points/cells being extracted.


In [ ]:
boundaries = mesh.extract_feature_edges(
    non_manifold_edges=True, feature_edges=False, manifold_edges=False
)
boundaries

The cells are `~pyvista.CellType.TRIANGLE`{.interpreted-text
role="attr"} cells. For example, the first cell


In [ ]:
mesh.get_cell(0).type

Structured quadrilateral mesh of Sphere
=======================================

The structure of the mesh can be important. Instead of a triangulated
mesh, it can be useful to have a structured mesh that has an i-j-k
ordering that allows for simplified cell connectivity.

The points are generated as a regular grid in spherical coordinates
using `pyvista.spherical_to_cartesian`{.interpreted-text role="func"}.
Here, we will used the convention that `theta` is the azimuthal angle,
similar to longitude on the globe. `phi` is the polar angle, similar to
latitude on the globe.


In [ ]:
radius = 0.5
ntheta = 9
nphi = 12
theta = np.linspace(0, 2 * np.pi, ntheta)
phi = np.linspace(0, np.pi, nphi)

r_, phi_, theta_ = np.meshgrid([radius], phi, theta, indexing='ij')
x, y, z = pv.spherical_to_cartesian(r_, phi_, theta_)
mesh = pv.StructuredGrid(x, y, z)

The mesh has `~pyvista.CellType.QUAD`{.interpreted-text role="attr"}
cells. The cells that look triangular at the poles are actually
degenerate quadrilaterals, i.e. two points are coincident at the pole,
as will be shown later.


In [ ]:
mesh.plot(show_edges=True)

The mesh is of type `pyvista.StructuredGrid`{.interpreted-text
role="class"}.


In [ ]:
mesh

The first cell is at the top pole, and it is a
`~pyvista.CellType.QUAD`{.interpreted-text role="attr"} cell.


In [ ]:
cell = mesh.get_cell(0)
cell.type

The first cell has two degenerate points.


In [ ]:
cell.points

The cells on either side of the \'seam\' along the start and end of the
azimuthal component are not connected. These can be detected by
extracting the boundary edges.


In [ ]:
boundaries = mesh.extract_feature_edges(
    non_manifold_edges=True, feature_edges=False, manifold_edges=False
)
boundaries

Visualize this by plotting the boundary edges of the mesh.


In [ ]:
pl = pv.Plotter()
pl.add_mesh(mesh, show_edges=True)
pl.add_mesh(boundaries, line_width=10, color='red')
pl.show()

Generate quadrilateral mesh of Sphere
=====================================

This example shows how a more complicated mesh can be defined.

In contrast to the example above, this example generates a mesh that
does not have degenerate points at the poles.
`~pyvista.CellType.TRIANGLE`{.interpreted-text role="attr"} cells will
be used at the poles. First, regenerate the structured data.


In [ ]:
radius = 0.5
ntheta = 9
nphi = 12
theta = np.linspace(0, 2 * np.pi, ntheta)
phi = np.linspace(0, np.pi, nphi)

We do not want duplicate points, so remove the duplicate in theta, which
results in 8 unique points in theta. Similarly, the poles at `phi=0` and
`phi=pi` will be handled separately to avoid duplicate points, which
results in 10 unique points in phi. Remove these from the grid in
spherical coordinates.


In [ ]:
theta = theta[:-1]
ntheta -= 1
phi = phi[1:-1]
nphi -= 2

Use `pyvista.spherical_to_cartesian`{.interpreted-text role="func"} to
generate cartesian coordinates for points in the `(N, 3)` format
required by PyVista. Note that this method results in the theta variable
changing the fastest.


In [ ]:
r_, phi_, theta_ = np.meshgrid([radius], phi, theta, indexing='ij')
x, y, z = pv.spherical_to_cartesian(r_, phi_, theta_)
points = np.vstack((x.ravel(), y.ravel(), z.ravel())).transpose()

The first and last points are the poles.


In [ ]:
points = np.insert(points, 0, [0.0, 0.0, radius], axis=0)
points = np.append(points, [[0.0, 0.0, -radius]], axis=0)

First we will generate the cell-point connectivity similar to the
previous examples. At the poles, we will form triangles with the pole
and two adjacent points from the closest ring of points at a given `phi`
position. Otherwise, we will form quadrilaterals between two adjacent
points on consecutive `phi` positions.

The first triangle in the mesh is point id `0`, i.e. the pole, and the
first two points at the first `phi` position, id\'s `1` and `2`. the
next triangle contains the pole again and the next set of points, id\'s
`2` and `3` and so on. The last point in the ring, id `8` connects to
the first point in the ring, `1`, to form the last triangle. Exclude it
from the loop and add separately.


In [ ]:
faces = []
for i in range(1, ntheta):
    faces.extend([3, 0, i, i + 1])

faces.extend([3, 0, ntheta, 1])

Demonstrate the connectivity of the mesh so far.


In [ ]:
points_to_label = tuple(range(ntheta + 1))
mesh = pv.PolyData(points, faces=faces)
pl = pv.Plotter()
pl.add_mesh(mesh, show_edges=True)
pl.add_point_labels(
    mesh.points[points_to_label, :], points_to_label, font_size=30, fill_shape=False
)
pl.view_xy()
pl.show()

Next form the quadrilaterals. This process is the same except by
connecting points across two levels of `phi`. For point `1` and point
`2`, these are connected to point `9` and point `10`. Note for
quadrilaterals it must be defined in a consistent direction. Again, the
last point(s) in the theta direction connect back to the first point(s).


In [ ]:
for i in range(1, ntheta):
    faces.extend([4, i, i + 1, i + ntheta + 1, i + ntheta])

faces.extend([4, ntheta, 1, ntheta + 1, ntheta * 2])

Demonstrate the connectivity of the mesh with first quad layer.


In [ ]:
points_to_label = tuple(range(ntheta * 2 + 1))
mesh = pv.PolyData(points, faces=faces)
pl = pv.Plotter()
pl.add_mesh(mesh, show_edges=True)
pl.add_point_labels(
    mesh.points[points_to_label, :],
    points_to_label,
    font_size=30,
    fill_shape=False,
    always_visible=True,
)
pl.view_xy()
pl.show()

Next we loop over all adjacent levels of phi to form all the
quadrilaterals and add the layer of triangles on the ending pole. Since
we already formed the first layer of quadrilaterals, let\'s start over
to make cleaner code.


In [ ]:
faces = []
for i in range(1, ntheta):
    faces.extend([3, 0, i, i + 1])

faces.extend([3, 0, ntheta, 1])

for j in range(nphi - 1):
    for i in range(1, ntheta):
        faces.extend(
            [4, j * ntheta + i, j * ntheta + i + 1, i + (j + 1) * ntheta + 1, i + (j + 1) * ntheta]
        )

    faces.extend([4, (j + 1) * ntheta, j * ntheta + 1, (j + 1) * ntheta + 1, (j + 2) * ntheta])

for i in range(1, ntheta):
    faces.extend([3, nphi * ntheta + 1, (nphi - 1) * ntheta + i, (nphi - 1) * ntheta + i + 1])

faces.extend([3, nphi * ntheta + 1, nphi * ntheta, (nphi - 1) * ntheta + 1])

We will use a `pyvista.PolyData`{.interpreted-text role="class"} mesh
here, but a `pyvista.UnstructuredGrid`{.interpreted-text role="class"}
could also be used.


In [ ]:
mesh = pv.PolyData(points, faces=faces)

This mesh is `manifold <pyvista.PolyData.is_manifold>`{.interpreted-text
role="func"} like `pyvista.Sphere`{.interpreted-text role="func"}. To
demonstrate this, there are no boundaries on the mesh as indicated by no
points/cells being extracted.


In [ ]:
boundaries = mesh.extract_feature_edges(
    non_manifold_edges=True, feature_edges=False, manifold_edges=False
)
boundaries

All the point labels are messy when plotted, so don\'t add to the final
plot.


In [ ]:
mesh.plot(show_edges=True)

::: {.tags}
load
:::
